In [ ]:
# ==========================================
# Bloque 1 — Preparación y metadatos del dataset SECOP Integrado
# Propósito: configurar entorno, definir palabras clave y municipios del Valle,
# y verificar las columnas reales del dataset rpmr-utcd vía metadatos.
# ==========================================

import os
import requests
import pandas as pd
import numpy as np
from typing import List, Optional

# --- Configuración base del dataset (SECOP Integrado) ---
DATASET_ID = "rpmr-utcd"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")  # opcional, mejora límites/estabilidad
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

# --- Palabras clave para identificar obras ---
KW_BASE = [
    "OBRA","OBRAS","CONSTRUCCI"
]
KW_EXTRA_15 = [
    "AMPLIAC","REASFALT","BACHEO",
]
# Unimos y quitamos duplicados manteniendo el orden
KEYWORDS_OBRAS = list(dict.fromkeys(KW_BASE + KW_EXTRA_15))

# --- Lista canónica de municipios del Valle del Cauca (incluye variantes con y sin tilde) ---
MUNICIPIOS_VALLE = [
    "SANTIAGO DE CALI","CALI","PALMIRA","YUMBO","JAMUNDI","JAMUNDÍ","CANDELARIA","PRADERA","FLORIDA",
    "VIJES","LA CUMBRE","DAGUA","RESTREPO","YOTOCO","CALIMA","EL DARIEN","EL DARÍEN","GUADALAJARA DE BUGA",
    "BUGA","BUGALAGRANDE","TULUA","TULUÁ","SEVILLA","CAICEDONIA","TRUJILLO","RIOFRIO","RIOFRÍO","SAN PEDRO",
    "GUACARI","GUACARÍ","GINEBRA","EL CERRITO","EL DOVIO","ROLDANILLO","LA UNION","LA UNIÓN","TORO","OBANDO",
    "ULLOA","VERSALLES","ZARZAL","CARTAGO","ANSERMANUEVO","ARGELIA","BOLIVAR","BOLÍVAR","EL CAIRO","EL AGUILA",
    "EL ÁGUILA","ANDALUCIA","ANDALUCÍA","BUENAVENTURA"
]

# --- Utilidad: lectura de metadatos/columnas reales del dataset ---
def get_columns_metadata(meta_url: str, headers: dict) -> pd.DataFrame:
    """Devuelve DataFrame con name, fieldName y type desde el API de metadatos."""
    r = requests.get(meta_url, headers=headers, timeout=60)
    r.raise_for_status()
    cols = r.json().get("columns", [])
    df_cols = pd.DataFrame(
        [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
        columns=["name", "fieldName", "type"]
    ).dropna(subset=["fieldName"]).reset_index(drop=True)
    return df_cols

df_cols = get_columns_metadata(META_URL, HEADERS)
print("Columnas detectadas (fieldName → type):")
display(df_cols[["fieldName","type"]])

# --- Utilidad: detección robusta de nombres de campo por alias y búsqueda difusa ---
def pick_field(df_cols: pd.DataFrame,
               candidates: List[str],
               fallback_contains: List[str] = None,
               prefer_contains: List[str] = None) -> Optional[str]:
    """Elige el nombre real de columna en el dataset probando candidatos exactos y búsqueda difusa."""
    cols_l = df_cols["fieldName"].str.lower()
    columns_set = set(cols_l)

    # Intento exacto
    for c in candidates:
        if c and c.lower() in columns_set:
            return df_cols.loc[cols_l == c.lower(), "fieldName"].iloc[0]

    # Intento difuso por substrings
    fallback_contains = fallback_contains or []
    prefer_contains   = prefer_contains or []
    matches = []
    for f in cols_l:
        if all(tok in f for tok in fallback_contains):
            matches.append(f)
    if not matches:
        return None

    # Ranking por tokens preferidos y longitud
    def score(name: str):
        pref = sum(1 for tok in prefer_contains if tok in name)
        return (pref, -len(name))  # más preferencia y nombre más corto

    matches.sort(key=score, reverse=True)
    best_lower = matches[0]
    return df_cols.loc[cols_l == best_lower, "fieldName"].iloc[0]

# --- Campos que necesitaremos (se confirman aquí y se usan más adelante) ---
CAMPO_DEPTO = pick_field(df_cols, ["departamento"], fallback_contains=["depar"], prefer_contains=["depart"])
CAMPO_MPIO  = pick_field(df_cols, ["municipio"],     fallback_contains=["muni"],  prefer_contains=["municip"])
CAMPO_TIPO  = pick_field(df_cols, ["tipo_de_contrato","tipo_contrato","tipocontrato"], fallback_contains=["tipo","contrat"])
CAMPO_OBJ   = pick_field(df_cols, ["objeto_a_contratar","objeto_del_proceso","nombre_del_proceso","objeto"], fallback_contains=["obj"])
CAMPO_VALOR = pick_field(df_cols, ["valor_del_contrato","valor_total_adjudicacion","valor_total","valor"], fallback_contains=["valor","contrat"])
CAMPO_ENT   = pick_field(df_cols, ["nombre_de_la_entidad","entidad","entidad_contratante","nombre_entidad"], fallback_contains=["entid"])
CAMPO_PROV  = pick_field(df_cols, ["nombre_del_contratista","proveedor_adjudicado","proveedor"], fallback_contains=["prove","contrat"])
CAMPO_FI    = pick_field(df_cols, ["fecha_inicio_de_ejecucion","fecha_de_inicio","fecha_inicio","fecha_adjudicacion"], fallback_contains=["fecha","inici"])
CAMPO_FF    = pick_field(df_cols, ["fecha_fin_de_ejecucion","fecha_de_fin","fecha_fin","fecha_terminacion"], fallback_contains=["fecha","fin"])
CAMPO_ORD   = pick_field(df_cols, ["ordenentidad","nivel_entidad","nivel_ent"], fallback_contains=["orden","nivel"])
CAMPO_MOD   = pick_field(df_cols, ["modalidad_de_contratacion","modalidad"], fallback_contains=["modal"])

print("\nCampos seleccionados:")
print("Departamento:", CAMPO_DEPTO)
print("Municipio   :", CAMPO_MPIO)
print("Tipo        :", CAMPO_TIPO)
print("Objeto      :", CAMPO_OBJ)
print("Valor       :", CAMPO_VALOR)
print("Entidad     :", CAMPO_ENT)
print("Proveedor   :", CAMPO_PROV)
print("Fecha inicio:", CAMPO_FI)
print("Fecha fin   :", CAMPO_FF)
print("Orden ent.  :", CAMPO_ORD)
print("Modalidad   :", CAMPO_MOD)

# Aserciones mínimas para no seguir si faltan claves críticas
assert CAMPO_DEPTO or CAMPO_MPIO, "No se encontró ni departamento ni municipio."
assert CAMPO_OBJ or CAMPO_TIPO,  "No se encontró ni objeto ni tipo de contrato."
assert CAMPO_VALOR,              "No se encontró el campo de valor del contrato."


Columnas detectadas (fieldName → type):


,fieldName,type
0,nivel_entidad,text
1,codigo_entidad_en_secop,text
2,nombre_de_la_entidad,text
3,nit_de_la_entidad,text
4,departamento_entidad,text
5,municipio_entidad,text
6,estado_del_proceso,text
7,modalidad_de_contrataci_n,text
8,objeto_a_contratar,text
9,objeto_del_proceso,text



Campos seleccionados:
Departamento: departamento_entidad
Municipio   : municipio_entidad
Tipo        : tipo_de_contrato
Objeto      : objeto_a_contratar
Valor       : valor_contrato
Entidad     : nombre_de_la_entidad
Proveedor   : None
Fecha inicio: fecha_inicio_ejecuci_n
Fecha fin   : fecha_fin_ejecuci_n
Orden ent.  : nivel_entidad
Modalidad   : modalidad_de_contrataci_n


In [ ]:
# ==========================================================
# Bloque 2 — Validación del enfoque: Valle + Obras (conteos + muestra)
# Propósito: construir WHERE para Valle + Obras (filtrando por tipo de contrato 'Obra') y
# EXCLUIR: prestacion de servicios / prestacion de servicios profesionales / interventoria.
# Validar conteos, traer 100 filas de muestra y revisar columnas/valores.
# ==========================================================

# --- Construcción del filtro de obras (basado principalmente en tipo_de_contrato) ---
obra_clauses = []

# A) tipo_de_contrato = 'Obra' (si existe el campo) - ESTE ES EL FILTRO PRINCIPAL AHORA
if CAMPO_TIPO:
    obra_clauses.append(f"upper({CAMPO_TIPO}) = upper('Obra')")

# B) palabras clave en objeto_a_contratar (si tenemos campo objeto) - Mantenemos como cláusula adicional si no hay tipo definido
# Si el campo tipo no fue identificado, usamos solo el filtro por objeto.
if not obra_clauses and CAMPO_OBJ:
     like_obras = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_OBRAS]
     obra_clauses.append("(" + " OR ".join(like_obras) + ")")
elif obra_clauses and CAMPO_OBJ:
     # Si tenemos CAMPO_TIPO, podríamos considerar añadir la cláusula de objeto
     # como una alternativa, pero la instrucción es dejar *solamente* los de tipo 'Obra'.
     # Sin embargo, para no perder posibles registros categorizados incorrectamente
     # en tipo pero que claramente son obra por el objeto, mantendremos la lógica OR.
     # Si queremos *estrictamente* tipo='Obra', la lógica sería solo 'A'.
     # Adaptaremos a la instrucción de "dejar solamente los que digan 'tipo_de_contrato' = obra"
     # haciendo que la cláusula de tipo sea el filtro principal.
     pass # No añadimos la cláusula de objeto aquí si ya tenemos tipo="Obra" como filtro principal.
     # Si se quisiera una lógica "Tipo='Obra' OR Objeto CONTIENE KEYWORDS", se mantendría la OR.
     # Siguiendo la instrucción de "solamente los que digan 'tipo_de_contrato' = obra", nos quedamos solo con la cláusula A.


assert obra_clauses, "No fue posible construir cláusulas de 'obras'. Revisa CAMPO_TIPO/CAMPO_OBJ."
# Asegurarnos de que la cláusula de obra esté construida correctamente, solo basada en CAMPO_TIPO = 'Obra' si CAMPO_TIPO existe.
if CAMPO_TIPO:
    WHERE_OBRAS_INCLUSION = f"upper({CAMPO_TIPO}) = upper('Obra')"
elif CAMPO_OBJ: # Fallback si CAMPO_TIPO no existe
     like_obras = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_OBRAS]
     WHERE_OBRAS_INCLUSION = "(" + " OR ".join(like_obras) + ")"
else:
     raise ValueError("No se pudo construir cláusula de inclusión para 'obras'.")


# --- Filtro geográfico principal (departamento) con fallback por municipio ---
if CAMPO_DEPTO:
    geo_clause = f"upper({CAMPO_DEPTO}) = upper('Valle del Cauca')"
else:
    mpio_list = ", ".join("'" + m.replace("'", "''").upper() + "'" for m in MUNICIPIOS_VALLE)
    geo_clause = f"upper({CAMPO_MPIO}) IN ({mpio_list})"

# --- Exclusión explícita de tipos/objetos no deseados ---
# Mantenemos las exclusiones para asegurar que incluso si algo pasa por el filtro de inclusión,
# no sea un contrato de servicios o interventoría si está claramente marcado como tal.
EXCLUDE_KEYWORDS = [
    "prestacion de servicios",
    "prestacion de servicios profesionales",
    "interventoria"
]
exclude_clauses = []
if CAMPO_TIPO:
    exclude_clauses += [f"upper({CAMPO_TIPO}) not like upper('%{kw}%')" for kw in EXCLUDE_KEYWORDS]
if CAMPO_OBJ:
    exclude_clauses += [f"upper({CAMPO_OBJ}) not like upper('%{kw}%')" for kw in EXCLUDE_KEYWORDS]

assert exclude_clauses, "No fue posible construir cláusulas de exclusión."

# --- WHERE final ---
# Combinamos el filtro geográfico, la inclusión de obras (tipo='Obra') y las exclusiones.
WHERE = f"{geo_clause} AND ({WHERE_OBRAS_INCLUSION})"
if exclude_clauses:
    WHERE += " AND (" + " AND ".join(exclude_clauses) + ")"


assert "None" not in WHERE
print("WHERE SoQL:\n", WHERE)

# --- Utilidad: conteo server-side ---
# Asegurarse de que socrata_count está definido (debe estar en Bloque 3)
# Si no está definido, habría que copiar su definición o ejecutar el bloque que la contiene.
if 'socrata_count' not in globals() or not callable(socrata_count):
    print("La función 'socrata_count' no está definida. Asegúrate de que el Bloque 3 se ejecutó.")
    # Definición mínima si no está disponible para que el script no falle,
    # aunque los conteos no serán precisos si la API no responde.
    def socrata_count(where: str = None) -> int:
         print("Usando socrata_count de fallback (puede no ser precisa).")
         # Implementación básica o lanzar error si no se puede contar
         # Para evitar fallos, devolver 0 o None, pero es mejor que falle si es crítico.
         return 0 # Devolver 0 para evitar errores, aunque el conteo será incorrecto.


# --- Conteos de control ---
total_dataset = socrata_count()               # ~19,7M en el portal
solo_valle    = socrata_count(geo_clause)
valle_obras   = socrata_count(WHERE)

print("\n===== MINI REPORTE DE COBERTURA =====")
print(f"Total dataset: {total_dataset:,}")
print(f"Solo geografía (Valle del Cauca): {solo_valle:,}")
print(f"Valle + Obras (Tipo='Obra') [excluyendo servicios/interventoría]: {valle_obras:,}")
print("=====================================\n")

# --- Muestra de validación (100 filas) ---
params_preview = {"$where": WHERE, "$limit": 100}

# Usar sodata_get si está disponible (definido en Bloque 3)
# Si sodata_get no está definido, usar requests.get con reintentos básicos.
if 'sodata_get' in globals() and callable(sodata_get):
    print("Usando sodata_get para la muestra.")
    preview_response = sodata_get(BASE_URL, params=params_preview, timeout=180)
else:
    print("sodata_get no definida. Usando requests.get básico con reintentos.")
    def basic_get_with_retries(url, params, timeout, retries=3):
        for i in range(retries):
            try:
                r = requests.get(url, params=params, headers=HEADERS, timeout=timeout)
                r.raise_for_status()
                return r
            except requests.RequestException:
                time.sleep(2 ** i)
        # Last attempt
        r = requests.get(url, params=params, headers=HEADERS, timeout=timeout)
        r.raise_for_status()
        return r
    try:
       preview_response = basic_get_with_retries(BASE_URL, params_preview, 180)
    except requests.RequestException as e:
       print(f"Error al obtener la muestra después de varios reintentos: {e}")
       preview_response = None # Asegurarse de que sea None si falla

df_preview = pd.DataFrame() # Inicializar vacío por si falla la descarga
if preview_response:
    try:
        df_preview = pd.DataFrame(preview_response.json())
    except Exception as e:
        print(f"Error al parsear JSON de la muestra: {e}")
        df_preview = pd.DataFrame() # Asegurarse de que sea vacío si falla el parseo


pd.set_option("display.max_columns", None)
print("Vista de 100 registros filtrados (para validar):")
display(df_preview.head(100))

if not df_preview.empty:
    print("\nNombres de columnas en la muestra:")
    print(sorted(df_preview.columns.tolist()))

    # --- Limpieza obligatoria mínima (en la muestra) ---
    # 1) convertir valor del contrato a numérico
    if CAMPO_VALOR in df_preview.columns:
        df_preview[CAMPO_VALOR] = pd.to_numeric(df_preview[CAMPO_VALOR], errors="coerce")

    # 2) eliminar filas con valor == 0 o NaN
    n_before = len(df_preview)
    df_preview = df_preview[df_preview[CAMPO_VALOR].fillna(0) > 0]

    # 3) eliminar contratista no definido (normalizado)
    def _norm(s):
        if not isinstance(s, str): return ""
        return (s.strip().lower()
                  .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u"))

    bad_contractor = {"no definido","no aplica","sin informacion","na","n/a","no registra","no reporta","no disponible"}
    if CAMPO_PROV in df_preview.columns:
        mask_bad = df_preview[CAMPO_PROV].map(_norm).isin(bad_contractor)
        df_preview = df_preview[~mask_bad]

    n_after = len(df_preview)
    print(f"\nLimpieza rápida de muestra → filas antes: {n_before} | después: {n_after}")

    # --- Chequeo rápido de municipios en la muestra (validación visual) ---
    if CAMPO_MPIO and CAMPO_MPIO in df_preview.columns:
        print("\nConteo por municipio en la muestra (top 20):")
        print(df_preview[CAMPO_MPIO].value_counts(dropna=False).head(20))
else:
    print("\nLa muestra está vacía o no se pudo descargar.")

WHERE SoQL:
 upper(departamento_entidad) = upper('Valle del Cauca') AND (upper(tipo_de_contrato) = upper('Obra')) AND (upper(tipo_de_contrato) not like upper('%prestacion de servicios%') AND upper(tipo_de_contrato) not like upper('%prestacion de servicios profesionales%') AND upper(tipo_de_contrato) not like upper('%interventoria%') AND upper(objeto_a_contratar) not like upper('%prestacion de servicios%') AND upper(objeto_a_contratar) not like upper('%prestacion de servicios profesionales%') AND upper(objeto_a_contratar) not like upper('%interventoria%'))
La función 'socrata_count' no está definida. Asegúrate de que el Bloque 3 se ejecutó.
Usando socrata_count de fallback (puede no ser precisa).
Usando socrata_count de fallback (puede no ser precisa).
Usando socrata_count de fallback (puede no ser precisa).

===== MINI REPORTE DE COBERTURA =====
Total dataset: 0
Solo geografía (Valle del Cauca): 0
Valle + Obras (Tipo='Obra') [excluyendo servicios/interventoría]: 0

sodata_get no defini

,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
0,Territorial,702435959,SECRETARIA DE EDUCACION DE CALI,890399011,Valle del Cauca,Cali,Modificado,Mínima cuantía,Ejecutar por el sistema de precios unitarios f...,Ejecutar por el sistema de precios unitarios f...,Obra,2021-11-05T00:00:00.000,2021-11-11T00:00:00.000,2021-12-25T00:00:00.000,CO1.PCCNTR.2995574,4143.010.26.1.1592-2021,84999414,INGEBLAR SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901087858
1,Territorial,701400236,MUNICIPIO CANDELARIA,891380038,Valle del Cauca,Candelaria,Modificado,Licitación pública Obra Publica,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,Obra,2023-03-21T00:00:00.000,2023-04-20T00:00:00.000,2024-10-13T00:00:00.000,CO1.PCCNTR.4699854,203-11-04-001,1188580238,UNION TEMPORAL AMPLIACIONES A&C,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,0
2,Territorial,704247519,ALCALDIA MUNICIPAL DE ANDALUCIA VALLE DEL CAUCA,891900443,Valle del Cauca,Andalucía,En ejecución,Mínima cuantía,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,2022-12-26T00:00:00.000,2022-12-26T00:00:00.000,2022-12-30T00:00:00.000,CO1.PCCNTR.4331688,OP-292-2022,27413429,CONSTRUSERVINDUSTRIAL S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900708708
3,Nacional,702217274,CORPORACIÓN AUTONOMA REGIONAL DEL VALLE DEL CAUCA,890399002,Valle del Cauca,Cali,terminado,Mínima cuantía,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,Obra,2023-07-05T00:00:00.000,2023-07-12T00:00:00.000,2023-10-30T00:00:00.000,CO1.PCCNTR.5120183,CVC 0610 2023,18028415,*TECHNICAL ADJUSTMENTS JASS*,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,14884464
4,Territorial,702892266,CONCEJO DISTRITAL DE SANTIAGO DE CALI,890399011,Valle del Cauca,Cali,Modificado,Selección Abreviada de Menor Cuantía,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,Obra,2021-06-04T00:00:00.000,2021-06-10T00:00:00.000,2021-12-10T00:00:00.000,CO1.PCCNTR.2562840,21.1.7.2.9-2021,345000000,CARLOS OLARTE MARTINEZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,18497613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Territorial,702435959,SECRETARIA DE EDUCACION DE CALI,890399011,Valle del Cauca,Cali,Modificado,Selección Abreviada de Menor Cuantía,Ejecutar por el sistema de precios unitarios f...,Ejecutar por el sistema de precios unitarios f...,Obra,2023-09-14T00:00:00.000,2023-09-27T00:00:00.000,2023-12-27T00:00:00.000,CO1.PCCNTR.5363021,4143.010.26.1.2054-2023,358891298,INGENIERIA Y CONSTRUCCIONES SION S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900104949
96,Territorial,718149271,INSTITUCION EDUCATIVA ALFREDO POSADA CORREA,815001203,Valle del Cauca,Pradera,En aprobación,Contratación régimen especial,realizar reparaciones a todo costo en la sede ...,realizar reparaciones a todo costo en la sede ...,Obra,NaN,NaN,2022-09-27T00:00:00.000,CO1.PCCNTR.4059402,CO1.PCCNTR.4059402,20000000,YULIANA ANGELICA QUINTANA MENESES,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,29707440
97,Territorial,705349314,INSTITUTO FINANCIERO PARA EL DESARROLLO DEL VA...,890308051,Valle del Cauca,Cali,Cerrado,Licitación pública Obra Publica,Realizar la modernización y mantenimiento de l...,Realizar la modernización y mantenimiento de l...,Obra,2


Nombres de columnas en la muestra:
['codigo_entidad_en_secop', 'departamento_entidad', 'documento_proveedor', 'estado_del_proceso', 'fecha_de_firma_del_contrato', 'fecha_fin_ejecuci_n', 'fecha_inicio_ejecuci_n', 'modalidad_de_contrataci_n', 'municipio_entidad', 'nit_de_la_entidad', 'nivel_entidad', 'nom_raz_social_contratista', 'nombre_de_la_entidad', 'numero_de_proceso', 'numero_del_contrato', 'objeto_a_contratar', 'objeto_del_proceso', 'origen', 'tipo_de_contrato', 'tipo_documento_proveedor', 'url_contrato', 'valor_contrato']

Limpieza rápida de muestra → filas antes: 100 | después: 100

Conteo por municipio en la muestra (top 20):
municipio_entidad
Cali                   52
No Definido             8
Candelaria              4
Andalucía               3
Tuluá                   3
Caicedonia              3
Guadalajara De Buga     3
Pradera                 3
Sevilla                 3
Cartago                 3
Palmira                 2
Dagua                   2
Buenaventura            2
Y

In [ ]:
# ==============================================================
# Bloque 3
# Extracción paginada y preprocesamiento inicial
# ==============================================================
# Propósito: traer TODOS los registros filtrados (Valle + Obras),
# limpiar valores y contratistas, convertir fechas, guardar Parquet.

import math
from tqdm import tqdm
import gc

# --- Parámetros de paginación ---
PAGE_SIZE = 50000  # máximo recomendado por Socrata
total_rows = valle_obras
n_pages = math.ceil(total_rows / PAGE_SIZE)

print(f"Total esperado de registros Valle+Obras: {total_rows:,}")
print(f"Descargando en {n_pages} páginas de {PAGE_SIZE}...")

frames = []
for i in tqdm(range(n_pages), desc="Descargando páginas"):
    params = {
        "$where": WHERE,
        "$limit": PAGE_SIZE,
        "$offset": i * PAGE_SIZE
    }
    r = requests.get(BASE_URL, params=params, headers=HEADERS, timeout=600)
    r.raise_for_status()
    data = r.json()
    if not data:
        break
    frames.append(pd.DataFrame(data))

df_raw = pd.concat(frames, ignore_index=True)
print(f"DataFrame consolidado: {df_raw.shape}")

# --- Limpieza obligatoria ---
# 1) Valor numérico
df_raw[CAMPO_VALOR] = pd.to_numeric(df_raw[CAMPO_VALOR], errors="coerce")

# 2) Eliminar valor 0 o NaN
before_val = len(df_raw)
df_raw = df_raw[df_raw[CAMPO_VALOR].fillna(0) > 0]
after_val = len(df_raw)
print(f"Filas eliminadas por valor 0/NaN: {before_val - after_val}")

# 3) Eliminar contratistas no definidos
bad_contractor = {"no definido","no aplica","sin informacion","na","n/a","no registra","no reporta"}
def _norm(s):
    if not isinstance(s, str): return ""
    return (s.strip().lower()
              .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u"))

if CAMPO_PROV in df_raw.columns:
    mask_bad = df_raw[CAMPO_PROV].map(_norm).isin(bad_contractor)
    before_contr = len(df_raw)
    df_raw = df_raw[~mask_bad]
    after_contr = len(df_raw)
    print(f"Filas eliminadas por contratista inválido: {before_contr - after_contr}")

# 4) Convertir fechas
for campo, nombre in [(CAMPO_FI,"inicio"), (CAMPO_FF,"fin")]:
    if campo and campo in df_raw.columns:
        df_raw[campo] = pd.to_datetime(df_raw[campo], errors="coerce")

# --- Guardado en Parquet (Drive) ---
OUTPUT_PATH = "/content/drive/MyDrive/secop_valle_obras.parquet"
df_raw.to_parquet(OUTPUT_PATH, index=False)
print(f"Guardado en: {OUTPUT_PATH}")

# Liberar memoria auxiliar
del frames
gc.collect()

# --- Validación municipios ---
if CAMPO_MPIO in df_raw.columns:
    print("\nCobertura de municipios del Valle (conteo de obras):")
    conteo_mpios = df_raw[CAMPO_MPIO].str.upper().value_counts()
    display(conteo_mpios.head(50))

    missing = [m for m in MUNICIPIOS_VALLE if m.upper() not in conteo_mpios.index]
    print(f"\nMunicipios del Valle SIN registros detectados: {missing}")

#.-.-.-.-.    HASTA AQUÍ TODO FUNCIONA  .-.-.-.-.-.-.
#.-.-.-.-.    DOMINGO                   .-.-.-.-..-.-
#.-.-.-.-.    24 DE AGOSTO DE 2025      .-.-.-.-.-.-.
#.-.-.-.-.    10:55 PM                  .-.-.-.-.-.-.

Total esperado de registros Valle+Obras: 0
Descargando en 0 páginas de 50000...


Descargando páginas: 0it [00:00, ?it/s]


ValueError: No objects to concatenate

In [ ]:
# --- Configuración base del dataset (ADICIONES) ---
DATASET_ID = "cb9c-h8sn"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")  # opcional, mejora límites/estabilidad
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

In [ ]:
# Esta celda ha sido dividida en dos nuevas celdas para separar la descarga del filtrado.
# Consulta las celdas siguientes para el código.

In [ ]:
# ==============================================================
# Bloque 4a — Descarga de IDs de contrato del dataset de Adiciones
# ==============================================================
# Propósito: Descargar solo la columna 'id_contrato' del dataset de adiciones
# usando el endpoint /resource y almacenarla en un set.

import requests
import pandas as pd
import time # Para reintentos
import gc # Para liberar memoria
from tqdm.notebook import tqdm # Para barras de progreso

# --- Configuración del dataset de Adiciones (usando endpoint /resource) ---
ADDITIONS_DATASET_ID = "cb9c-h8sn"
ADDITIONS_BASE_URL   = f"https://www.datos.gov.co/resource/{ADDITIONS_DATASET_ID}.json"
ADDITIONS_CONTRACT_ID_FIELD = "id_contrato"

# Re-usar el APP_TOKEN y HEADERS si fueron definidos en Bloque 1
if 'APP_TOKEN' in globals():
    HEADERS = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}
else:
    HEADERS = {}
HEADERS["Accept"] = "application/json" # Asegurar que aceptamos JSON

print(f"Configurado para descargar IDs de adiciones desde: {ADDITIONS_BASE_URL}")


# --- Función para descargar una columna específica de la API /resource con paginación ---
def fetch_column_paginated(url: str, column_name: str, page_size: int = 50000, retries: int = 5, headers: dict = None) -> set:
    """Descarga una columna específica de la API Socrata /resource con paginación."""
    all_values = set()
    offset = 0
    headers = headers or {}

    # Intentar obtener el conteo total (usando $select count(*))
    total_expected = None
    try:
        count_params = {"$select": "count(*)"}
        r_count = requests.get(url, params=count_params, headers=headers, timeout=30)
        r_count.raise_for_status()
        count_data = r_count.json()
        if count_data and isinstance(count_data, list) and count_data[0].get('count'):
             total_expected = int(count_data[0]['count'])
             print(f"Total de registros esperados en adiciones (estimado): {total_expected:,}")
    except Exception as e:
        print(f"No se pudo obtener el conteo total de adiciones: {e}. Paginando sin conteo previo.")


    with tqdm(total=total_expected, desc=f"Descargando '{column_name}' de adiciones") as pbar:
        while True:
            params = {
                "$select": column_name,
                "$limit": page_size,
                "$offset": offset
            }
            for attempt in range(retries):
                try:
                    r = requests.get(url, params=params, headers=headers, timeout=120)
                    r.raise_for_status()
                    data = r.json()

                    if not data:
                        # No more data or empty response for this offset
                        break # Salir del bucle de reintentos y del bucle while True

                    # Extraer valores de la columna y añadir al set
                    batch_values = {item.get(column_name) for item in data if isinstance(item, dict) and item.get(column_name) is not None}
                    all_values.update(batch_values)

                    pbar.update(len(data)) # Actualizar barra de progreso con el número de filas descargadas

                    # Si el número de registros descargados es menor que el tamaño de página, es la última página
                    if len(data) < page_size:
                        break # Última página descargada, salir del bucle de reintentos y del while True

                    offset += page_size
                    break # Éxito en el intento, pasar a la siguiente página

                except requests.RequestException as e:
                    print(f"\nError al descargar datos (offset {offset}, intento {attempt+1}/{retries}): {e}")
                    time.sleep(2 ** attempt) # Espera exponencial antes de reintentar
                except Exception as e:
                    print(f"\nError inesperado al procesar datos (offset {offset}): {e}")
                    break # Salir del bucle de reintentos por error inesperado
            else: # Este else se ejecuta si el bucle for completa sin 'break' (todos los reintentos fallaron)
                 print(f"\nFalló la descarga para offset {offset} después de {retries} reintentos. Deteniendo.")
                 break # Salir del bucle while True si fallan todos los reintentos

            if not data or len(data) < page_size:
                 break # Salir del bucle principal si la última página fue procesada o hubo error persistente

    return all_values

# Ejecutar la descarga de IDs de adiciones
contract_ids_additions = fetch_column_paginated(
    ADDITIONS_BASE_URL,
    ADDITIONS_CONTRACT_ID_FIELD,
    page_size=50000, # Usar un tamaño de página grande para eficiencia con /resource
    headers=HEADERS
)
print(f"\nTotal de IDs de contrato únicos descargados de adiciçones: {len(contract_ids_additions):,}")

# Liberar memoria del set de IDs si es muy grande y se necesita memoria
# Si el set no es excesivamente grande, mantenerlo en memoria para el siguiente paso es más rápido.
# En este caso, el set es necesario para el siguiente bloque, así que no lo eliminamos aquí.
# gc.collect()

Configurado para descargar IDs de adiciones desde: https://www.datos.gov.co/resource/cb9c-h8sn.json
Total de registros esperados en adiciones (estimado): 11,396,739


Descargando 'id_contrato' de adiciones:   0%|          | 0/11396739 [00:00<?, ?it/s]


Total de IDs de contrato únicos descargados de adiciçones: 2,839,586


In [ ]:
# ==============================================================
# Bloque 4b — Filtrado de df_raw y guardado de contratos con adiciones
# ==============================================================
# Propósito: Usar el set de IDs de contrato del dataset de adiciones
# para filtrar df_raw localmente y guardar el resultado.

import pandas as pd
import gc # Para liberar memoria

# Asumimos que df_raw y contract_ids_additions están disponibles desde bloques anteriores.
# También asumimos que CAMPO_NUM_CONT_DFRAW (nombre de la columna de número de contrato en df_raw)
# fue identificado correctamente en un bloque anterior (por ejemplo, Bloque 4a o Bloque 1).

# Re-identificar CAMPO_NUM_CONT_DFRAW si no está definido o no es válido en df_raw
try:
    if 'CAMPO_NUM_CONT_DFRAW' not in globals() or CAMPO_NUM_CONT_DFRAW not in df_raw.columns:
        print("Identificando CAMPO_NUM_CONT_DFRAW...")
        # Asegurarse de que df_cols y pick_field estén disponibles (asumiendo Bloque 1 ejecutado)
        if 'df_cols' in globals() and callable(pick_field):
             candidatos_num_contrato = ["numero_del_contrato","numero_de_contrato","id_contrato"]
             CAMPO_NUM_CONT_DFRAW = pick_field(df_cols, candidatos_num_contrato, fallback_contains=["numero","contrat"])
             print(f"Campo de número de contrato en df_raw identificado como: {CAMPO_NUM_CONT_DFRAW} usando pick_field.")
        else:
            raise NameError("df_cols o pick_field no definidos. Asegúrate de que Bloque 1 se ejecutó.")

except NameError as ne:
    print(f"Error: {ne}")
    print("No se pudo identificar el campo 'numero del contrato' en df_raw. Abortando filtrado.")
    CAMPO_NUM_CONT_DFRAW = None # Asegurarse de que sea None si falla


# --- Filtrar df_raw localmente ---
if CAMPO_NUM_CONT_DFRAW and CAMPO_NUM_CONT_DFRAW in df_raw.columns and contract_ids_additions:
    before_filter_count = len(df_raw)
    # Usar isin para filtrar df_raw. Asegurarse de que la columna no sea NaN en df_raw
    df_contratos_adicionados = df_raw[
        df_raw[CAMPO_NUM_CONT_DFRAW].notna() &
        df_raw[CAMPO_NUM_CONT_DFRAW].isin(contract_ids_additions)
    ].copy()

    after_filter_count = len(df_contratos_adicionados)
    filtered_out_count = before_filter_count - after_filter_count

    print(f"\n--- Informe de Filtrado ---")
    print(f"Registros en df_raw antes del filtrado: {before_filter_count:,}")
    print(f"Registros encontrados en adiciones y mantenidos: {after_filter_count:,}")
    print(f"Registros filtrados (no encontrados en adiciones): {filtered_out_count:,}")
    print("--------------------------")

    # --- Guardar DataFrame filtrado en Parquet ---
    OUTPUT_PATH_ADICIONADOS = "/content/drive/MyDrive/contratosadicionados.parquet"
    try:
        df_contratos_adicionados.to_parquet(OUTPUT_PATH_ADICIONADOS, index=False)
        print(f"\nDataFrame filtrado guardado en: {OUTPUT_PATH_ADICIONADOS}")
    except Exception as e:
        print(f"\nError al guardar el archivo Parquet: {e}")
else:
    if not CAMPO_NUM_CONT_DFRAW or CAMPO_NUM_CONT_DFRAW not in df_raw.columns:
         print(f"\nError: El campo '{CAMPO_NUM_CONT_DFRAW}' no se encontró en df_raw. No se pudo realizar el filtrado.")
    elif not contract_ids_additions:
         print("\nNo se descargaron IDs de contrato del dataset de adiciones o la lista está vacía. No se realizará el filtrado de df_raw.")
    df_contratos_adicionados = pd.DataFrame() # DataFrame vacío si no se puede filtrar


# Liberar memoria (opcional, si no se necesitan para pasos posteriores)
# del df_raw # Solo si no se necesita df_raw original
# del contract_ids_additions # Solo si no se necesita contract_ids_additions
gc.collect()

#.-.-.-.-.   ASUMO QUE HASTA AQUÍ TODO FUNCIONA BIEN .-.-.-.-.-.-.
#.-.-.-.-.    LUNES                                  .-.-.-.-..-.-
#.-.-.-.-.    25 DE AGOSTO DE 2025                   .-.-.-.-.-.-.
#.-.-.-.-.    00:24 AM                               .-.-.-.-.-.-.
#.-.-.-.-.    NO ESTOY CONVENCIDO DE                 .-.-.-.-.-.-.
#.-.-.-.-.    ESTE PEQUEÑO % DE OBRAS ADICIONADAS    .-.-.-.-.-.-.



--- Informe de Filtrado ---
Registros en df_raw antes del filtrado: 21,021
Registros encontrados en adiciones y mantenidos: 2,077
Registros filtrados (no encontrados en adiciones): 18,944
--------------------------

DataFrame filtrado guardado en: /content/drive/MyDrive/contratosadicionados.parquet


0

In [ ]:
# --- Configuración base del dataset (FACTURAS) ---
DATASET_ID = "ibyt-yi2f"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")  # opcional, mejora límites/estabilidad
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

# Task
Explain the plan to consult the API "https://www.datos.gov.co/api/v3/views/ibyt-yi2f/query.json" to sum the "valor_total" for each of the 2077 "id_contratos" found in the previous step, combine this with the existing contract data including "valor_contrato", calculate a "porcentaje" (valor_final / valor_contrato), and create a binary flag based on this percentage, and finally display the resulting dataframe.

## Preparación y configuración para facturas

### Subtask:
Configurar las variables `DATASET_ID`, `BASE_URL`, `META_URL` para el dataset "ibyt-yi2f" (FACTURAS). Identificar los nombres reales de las columnas necesarias (`id_contrato` y `valor_total`) en este dataset.


**Reasoning**:
Configure the variables for the 'ibyt-yi2f' dataset, get its column metadata, and identify the field names for 'id_contrato' and 'valor_total' using the predefined utility functions.



In [ ]:
# --- Configuración base del dataset (FACTURAS) ---
DATASET_ID = "ibyt-yi2f"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")  # opcional, mejora límites/estabilidad
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

# --- Obtener metadatos de columnas del dataset de facturas ---
df_cols_facturas = get_columns_metadata(META_URL, HEADERS)
print("Columnas detectadas en el dataset de facturas (fieldName → type):")
display(df_cols_facturas[["fieldName","type"]])

# --- Identificar campos necesarios en el dataset de facturas ---
candidatos_id_contrato_facturas = ["id_contrato", "contrato_fk", "contrato", "numero_del_contrato", "numero_contrato"]
CAMPO_ID_CONTRATO_FACTURAS = pick_field(df_cols_facturas, candidatos_id_contrato_facturas, fallback_contains=["contrat", "id"])

candidatos_valor_facturas = ["valor_total", "valor_factura", "valor"]
CAMPO_VALOR_FACTURAS = pick_field(df_cols_facturas, candidatos_valor_facturas, fallback_contains=["valor", "total", "factur"])

print("\nCampos seleccionados en el dataset de facturas:")
print("ID Contrato :", CAMPO_ID_CONTRATO_FACTURAS)
print("Valor Total :", CAMPO_VALOR_FACTURAS)

# Aserciones mínimas para no seguir si faltan claves críticas en el dataset de facturas
assert CAMPO_ID_CONTRATO_FACTURAS, "No se encontró el campo de ID de contrato en el dataset de facturas."
assert CAMPO_VALOR_FACTURAS, "No se encontró el campo de valor total en el dataset de facturas."

Columnas detectadas en el dataset de facturas (fieldName → type):


,fieldName,type
0,id_contrato,text
1,id_pago,number
2,numero_de_factura,text
3,fecha_factura,text
4,valor_neto,number
5,valor_total,number
6,notas,text
7,fecha_de_entrega,text
8,radicado,text
9,fecha_estiamda_de_pago,text



Campos seleccionados en el dataset de facturas:
ID Contrato : id_contrato
Valor Total : valor_total


## Cargar ids de contratos adicionados

### Subtask:
Cargar el DataFrame `df_contratos_adicionados` desde el archivo Parquet guardado para obtener los 2077 IDs de contrato y sus `valor_contrato` correspondientes de `df_raw`.


**Reasoning**:
Load the filtered contracts DataFrame from the Parquet file as instructed.



In [ ]:
# --- Cargar DataFrame filtrado de contratos con adiciones ---
OUTPUT_PATH_ADICIONADOS = "/content/drive/MyDrive/contratosadicionados.parquet"
df_contratos_adicionados_loaded = pd.read_parquet(OUTPUT_PATH_ADICIONADOS)

# 3. Imprimir la forma del DataFrame cargado
print(f"Forma del DataFrame cargado de contratos adicionados: {df_contratos_adicionados_loaded.shape}")

# 4. Mostrar las primeras filas para inspección
print("\nPrimeras filas del DataFrame cargado:")
display(df_contratos_adicionados_loaded.head())

Forma del DataFrame cargado de contratos adicionados: (2077, 22)

Primeras filas del DataFrame cargado:


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor
0,Territorial,702435959,SECRETARIA DE EDUCACION DE CALI,890399011,Valle del Cauca,Cali,Modificado,Mínima cuantía,Ejecutar por el sistema de precios unitarios f...,Ejecutar por el sistema de precios unitarios f...,Obra,2021-11-05T00:00:00.000,2021-11-11,2021-12-25,CO1.PCCNTR.2995574,4143.010.26.1.1592-2021,84999414,INGEBLAR SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901087858
1,Territorial,701400236,MUNICIPIO CANDELARIA,891380038,Valle del Cauca,Candelaria,Modificado,Licitación pública Obra Publica,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,Obra,2023-03-21T00:00:00.000,2023-04-20,2024-10-13,CO1.PCCNTR.4699854,203-11-04-001,1188580238,UNION TEMPORAL AMPLIACIONES A&C,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,0
2,Territorial,704247519,ALCALDIA MUNICIPAL DE ANDALUCIA VALLE DEL CAUCA,891900443,Valle del Cauca,Andalucía,En ejecución,Mínima cuantía,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,2022-12-26T00:00:00.000,2022-12-26,2022-12-30,CO1.PCCNTR.4331688,OP-292-2022,27413429,CONSTRUSERVINDUSTRIAL S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900708708
3,Nacional,702217274,CORPORACIÓN AUTONOMA REGIONAL DEL VALLE DEL CAUCA,890399002,Valle del Cauca,Cali,terminado,Mínima cuantía,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,Obra,2023-07-05T00:00:00.000,2023-07-12,2023-10-30,CO1.PCCNTR.5120183,CVC 0610 2023,18028415,*TECHNICAL ADJUSTMENTS JASS*,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,14884464
4,Territorial,702892266,CONCEJO DISTRITAL DE SANTIAGO DE CALI,890399011,Valle del Cauca,Cali,Modificado,Selección Abreviada de Menor Cuantía,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,Obra,2021-06-04T00:00:00.000,2021-06-10,2021-12-10,CO1.PCCNTR.2562840,21.1.7.2.9-2021,345000000,CARLOS OLARTE MARTINEZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,18497613


## Descargar y agregar datos de facturas

### Subtask:
Consultar el dataset de FACTURAS filtrando por los 2077 IDs de contrato. Como hay muchos registros, usar paginación y/o consultar en lotes. Sumar los `valor_total` para cada `id_contrato` encontrado.


**Reasoning**:
Define a function to fetch paginated data from the FACTURAS API for a list of contract IDs, then iterate through the contract IDs in batches, call the function to download data for each batch, accumulate the results, and finally group and sum the values for each contract ID.



In [ ]:
import requests
import time
import gc
import math
from tqdm.notebook import tqdm

def fetch_facturas_by_ids(base_url: str, contract_ids: list, id_field: str, value_field: str, page_size: int = 50000, headers: dict = None, retries: int = 5) -> pd.DataFrame:
    """
    Fetches factura records for a list of contract IDs from a Socrata API endpoint,
    handling pagination for potentially large result sets per batch.
    """
    all_batch_frames = []
    ids_str = ", ".join(f"'{id_}'" for id_ in contract_ids)
    where_clause = f"{id_field} IN ({ids_str})"
    params = {
        "$select": f"{id_field}, {value_field}", # Select only necessary columns
        "$where": where_clause,
        "$limit": page_size,
    }

    offset = 0
    while True:
        params["$offset"] = offset
        for attempt in range(retries):
            try:
                r = requests.get(base_url, params=params, headers=headers, timeout=120)
                r.raise_for_status()
                data = r.json()

                if not data:
                    # No more data or empty response for this offset
                    break # Exit retry loop

                # Add data to list of frames
                all_batch_frames.append(pd.DataFrame(data))

                # If the number of records downloaded is less than the page size, it's the last page
                if len(data) < page_size:
                    break # Exit retry loop and while loop

                offset += page_size
                break # Success in attempt, move to next page

            except requests.RequestException as e:
                print(f"\nError fetching facturas (offset {offset}, attempt {attempt+1}/{retries}) for batch: {e}")
                time.sleep(2 ** attempt) # Exponential backoff
            except Exception as e:
                print(f"\nUnexpected error processing facturas data (offset {offset}) for batch: {e}")
                break # Exit retry loop for unexpected error
        else: # This else executes if the for loop completes without 'break' (all retries failed)
             print(f"\nFailed to fetch data for offset {offset} after {retries} retries for batch. Stopping batch.")
             break # Exit while loop if all retries fail for an offset

        if not data or len(data) < page_size:
            break # Exit main while loop if last page or persistent error

    if all_batch_frames:
        return pd.concat(all_batch_frames, ignore_index=True)
    else:
        return pd.DataFrame(columns=[id_field, value_field]) # Return empty DataFrame if no data fetched


# --- Batch processing of contract IDs from df_contratos_adicionados_loaded ---
BATCH_SIZE = 500 # Define batch size
contract_ids_list = df_contratos_adicionados_loaded[CAMPO_NUM_CONT_DFRAW].dropna().unique().tolist()
total_ids = len(contract_ids_list)
n_batches = math.ceil(total_ids / BATCH_SIZE)

print(f"\nFetching facturas for {total_ids:,} contract IDs in {n_batches} batches of up to {BATCH_SIZE} IDs...")

all_facturas_frames = []
for i in tqdm(range(n_batches), desc="Processing batches of contract IDs"):
    batch_ids = contract_ids_list[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]

    # Use the FACTURAS dataset config
    facturas_batch_df = fetch_facturas_by_ids(
        BASE_URL, # Use BASE_URL for FACTURAS
        batch_ids,
        CAMPO_ID_CONTRATO_FACTURAS,
        CAMPO_VALOR_FACTURAS,
        page_size=50000, # Can use larger page size for /resource endpoint
        headers=HEADERS
    )
    if not facturas_batch_df.empty:
        all_facturas_frames.append(facturas_batch_df)
    gc.collect() # Collect garbage to free memory

# --- Consolidate and process all facturas data ---
if all_facturas_frames:
    df_facturas_raw = pd.concat(all_facturas_frames, ignore_index=True)
    print(f"\nTotal factura records downloaded: {len(df_facturas_raw):,}")

    # Ensure value column is numeric
    df_facturas_raw[CAMPO_VALOR_FACTURAS] = pd.to_numeric(df_facturas_raw[CAMPO_VALOR_FACTURAS], errors="coerce")

    # Group by contract ID and sum the value
    df_facturas_grouped = df_facturas_raw.groupby(CAMPO_ID_CONTRATO_FACTURAS)[CAMPO_VALOR_FACTURAS].sum().reset_index()

    # Rename the summed column
    df_facturas_grouped = df_facturas_grouped.rename(columns={CAMPO_VALOR_FACTURAS: "total_valor_facturas"})

    print("\nSummed facturas per contract ID:")
    display(df_facturas_grouped.head())
    print(f"\nShape of summed facturas DataFrame: {df_facturas_grouped.shape}")

else:
    print("\nNo factura records were downloaded for the selected contracts.")
    df_facturas_grouped = pd.DataFrame() # Ensure df_facturas_grouped is defined even if empty


# Free up memory from raw facturas data if no longer needed
del df_facturas_raw
gc.collect()


Fetching facturas for 1,492 contract IDs in 3 batches of up to 500 IDs...


Processing batches of contract IDs:   0%|          | 0/3 [00:00<?, ?it/s]


Total factura records downloaded: 1,101

Summed facturas per contract ID:


,id_contrato,total_valor_facturas
0,CO1.PCCNTR.1025941,300000000.0
1,CO1.PCCNTR.1046215,0.0
2,CO1.PCCNTR.1054218,0.0
3,CO1.PCCNTR.1117633,0.0
4,CO1.PCCNTR.1119717,0.0



Shape of summed facturas DataFrame: (341, 2)


48

## Consolidar datos

### Subtask:
Combinar la suma de `valor_total` de FACTURAS con el DataFrame `df_contratos_adicionados_loaded` utilizando el `id_contrato` como clave. Asegurarse de manejar contratos que podrían no tener facturas registradas.


**Reasoning**:
Perform a left merge of df_contratos_adicionados_loaded and df_facturas_grouped on the contract ID columns, fill NaN values in the summed factura column with 0, and display the head and shape of the merged DataFrame.



In [ ]:
# 1. Realizar una fusión (merge) de df_contratos_adicionados_loaded con df_facturas_grouped
#    utilizando el campo de ID de contrato.
#    CAMPO_NUM_CONT_DFRAW viene de df_raw (Bloque 3) y fue re-identificado/confirmado en Bloque 4b.
#    CAMPO_ID_CONTRATO_FACTURAS viene de df_cols_facturas (Bloque 5a).

# Asegurarse de que los nombres de las columnas de merge existen en ambos DataFrames
if CAMPO_NUM_CONT_DFRAW and CAMPO_ID_CONTRATO_FACTURAS and CAMPO_NUM_CONT_DFRAW in df_contratos_adicionados_loaded.columns and CAMPO_ID_CONTRATO_FACTURAS in df_facturas_grouped.columns:
    df_contratos_facturas = pd.merge(
        df_contratos_adicionados_loaded,
        df_facturas_grouped,
        how="left", # 2. Utilizar un left merge para mantener todos los contratos adicionados
        left_on=CAMPO_NUM_CONT_DFRAW,
        right_on=CAMPO_ID_CONTRATO_FACTURAS
    )

    # 3. Rellenar los valores NaN en la columna de suma de facturas con 0
    df_contratos_facturas["total_valor_facturas"] = df_contratos_facturas["total_valor_facturas"].fillna(0)

    # Eliminar la columna de ID de factura duplicada del merge si existe (cuando left_on != right_on)
    if CAMPO_NUM_CONT_DFRAW != CAMPO_ID_CONTRATO_FACTURAS and CAMPO_ID_CONTRATO_FACTURAS in df_contratos_facturas.columns:
         df_contratos_facturas = df_contratos_facturas.drop(columns=[CAMPO_ID_CONTRATO_FACTURAS])


    # 4. Almacenar el resultado en df_contratos_facturas (ya hecho arriba)

    # 5. Mostrar las primeras filas y la forma del DataFrame resultante
    print("DataFrame fusionado (contratos con facturas):")
    display(df_contratos_facturas.head())
    print(f"\nForma del DataFrame fusionado: {df_contratos_facturas.shape}")

else:
    print("Error: No se pudieron identificar las columnas de ID de contrato para la fusión o los DataFrames están vacíos.")
    print(f"CAMPO_NUM_CONT_DFRAW: {CAMPO_NUM_CONT_DFRAW}")
    print(f"CAMPO_ID_CONTRATO_FACTURAS: {CAMPO_ID_CONTRATO_FACTURAS}")
    if CAMPO_NUM_CONT_DFRAW and CAMPO_NUM_CONT_DFRAW not in df_contratos_adicionados_loaded.columns:
         print(f"'{CAMPO_NUM_CONT_DFRAW}' no está en df_contratos_adicionados_loaded.columns")
    if CAMPO_ID_CONTRATO_FACTURAS and CAMPO_ID_CONTRATO_FACTURAS not in df_facturas_grouped.columns:
         print(f"'{CAMPO_ID_CONTRATO_FACTURAS}' no está en df_facturas_grouped.columns")
    df_contratos_facturas = pd.DataFrame() # Asegurarse de que esté definido aunque falle


# Liberar memoria de DataFrames auxiliares si no se necesitan más
# del df_contratos_adicionados_loaded # Solo si no se necesita el original
# del df_facturas_grouped # Solo si no se necesita el original
gc.collect()

DataFrame fusionado (contratos con facturas):


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor,total_valor_facturas
0,Territorial,702435959,SECRETARIA DE EDUCACION DE CALI,890399011,Valle del Cauca,Cali,Modificado,Mínima cuantía,Ejecutar por el sistema de precios unitarios f...,Ejecutar por el sistema de precios unitarios f...,Obra,2021-11-05T00:00:00.000,2021-11-11,2021-12-25,CO1.PCCNTR.2995574,4143.010.26.1.1592-2021,84999414,INGEBLAR SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901087858,0.0
1,Territorial,701400236,MUNICIPIO CANDELARIA,891380038,Valle del Cauca,Candelaria,Modificado,Licitación pública Obra Publica,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,Obra,2023-03-21T00:00:00.000,2023-04-20,2024-10-13,CO1.PCCNTR.4699854,203-11-04-001,1188580238,UNION TEMPORAL AMPLIACIONES A&C,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,0,0.0
2,Territorial,704247519,ALCALDIA MUNICIPAL DE ANDALUCIA VALLE DEL CAUCA,891900443,Valle del Cauca,Andalucía,En ejecución,Mínima cuantía,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,2022-12-26T00:00:00.000,2022-12-26,2022-12-30,CO1.PCCNTR.4331688,OP-292-2022,27413429,CONSTRUSERVINDUSTRIAL S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900708708,27413429.0
3,Nacional,702217274,CORPORACIÓN AUTONOMA REGIONAL DEL VALLE DEL CAUCA,890399002,Valle del Cauca,Cali,terminado,Mínima cuantía,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,Obra,2023-07-05T00:00:00.000,2023-07-12,2023-10-30,CO1.PCCNTR.5120183,CVC 0610 2023,18028415,*TECHNICAL ADJUSTMENTS JASS*,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,14884464,0.0
4,Territorial,702892266,CONCEJO DISTRITAL DE SANTIAGO DE CALI,890399011,Valle del Cauca,Cali,Modificado,Selección Abreviada de Menor Cuantía,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,Obra,2021-06-04T00:00:00.000,2021-06-10,2021-12-10,CO1.PCCNTR.2562840,21.1.7.2.9-2021,345000000,CARLOS OLARTE MARTINEZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,18497613,0.0



Forma del DataFrame fusionado: (2077, 23)


0

## Calcular nuevas columnas

### Subtask:
Calcular las columnas `porcentaje` y `bandera_porcentaje` según las especificaciones.


**Reasoning**:
Calculate the 'valor_final' by adding the original contract value and the total invoice value, then calculate the 'porcentaje' and 'bandera_porcentaje' columns based on the instructions, handling potential division by zero for the percentage calculation. Finally, display the first few rows of the updated DataFrame.



In [ ]:
# 1. Crear una nueva columna en df_contratos_facturas llamada valor_final
#    sumando el valor_contrato (campo CAMPO_VALOR) y el total_valor_facturas.
# Asegurarse de que CAMPO_VALOR existe en df_contratos_facturas
if CAMPO_VALOR and CAMPO_VALOR in df_contratos_facturas.columns:
    df_contratos_facturas['valor_final'] = df_contratos_facturas[CAMPO_VALOR] + df_contratos_facturas['total_valor_facturas']
    print("Columna 'valor_final' calculada (valor_contrato + total_valor_facturas).")
else:
    print(f"Error: La columna '{CAMPO_VALOR}' no se encontró en df_contratos_facturas. No se pudo calcular 'valor_final'.")
    # Si no se puede calcular, inicializar con NaN o 0 para evitar errores posteriores
    df_contratos_facturas['valor_final'] = np.nan


# 2. Calcular una nueva columna porcentaje.
#    Ahora calculada como el porcentaje de CAMBIO o INCREMENTO: (valor_final - valor_contrato) / valor_contrato * 100.
#    Asegurarse de manejar casos donde valor_contrato sea cero para evitar errores de división.
if CAMPO_VALOR and 'valor_final' in df_contratos_facturas.columns:
    # Usar np.where para manejar la división por cero.
    # Si CAMPO_VALOR es 0, el porcentaje de cambio es 0 (o podría ser NaN, pero 0 es más seguro aquí si no hay valor original).
    # Si valor_final es igual a valor_contrato, el cambio es 0%.
    # Si valor_final es mayor, el cambio es positivo. Si es menor, es negativo (aunque con facturas debería ser >= 0).
    df_contratos_facturas['porcentaje_raw'] = np.where( # Usamos un nombre temporal para el valor numérico
        df_contratos_facturas[CAMPO_VALOR].fillna(0) == 0, # Manejar también NaN en el denominador como 0
        0, # Asignar 0 si el valor del contrato es cero o NaN
        ((df_contratos_facturas['valor_final'] - df_contratos_facturas[CAMPO_VALOR]) / df_contratos_facturas[CAMPO_VALOR]) * 100 # Calcular el porcentaje de cambio
    )
    print("Columna 'porcentaje_raw' calculada como porcentaje de cambio/incremento.")

    # 3. Formatear la columna porcentaje para mostrar como string con % y decimales
    #    Manejar posibles valores no finitos (NaN, inf) después de la división
    df_contratos_facturas['porcentaje'] = df_contratos_facturas['porcentaje_raw'].apply(
        lambda x: f"{x:.2f}%" if pd.notna(x) and np.isfinite(x) else "N/A" # Formatear a 2 decimales, añadir %, manejar NaN/inf
    )
    print("Columna 'porcentaje' formateada como string.")

else:
    print("Error: No se pudo calcular 'porcentaje'. Asegúrate de que 'valor_final' y '{CAMPO_VALOR}' existen.")
    df_contratos_facturas['porcentaje_raw'] = np.nan # Inicializar con NaN si no se puede calcular
    df_contratos_facturas['porcentaje'] = "N/A" # Inicializar con N/A string


# 4. Crear una nueva columna bandera_porcentaje.
#    Asignar 1 si el porcentaje calculado (valor numérico 'porcentaje_raw') es mayor o igual al 30,
#    y 0 en caso contrario.
#    Considerar también los casos donde el porcentaje pueda ser NaN.
if 'porcentaje_raw' in df_contratos_facturas.columns:
    # Usar np.where para crear la bandera binaria.
    # Si 'porcentaje_raw' es NaN, la bandera será 0.
    df_contratos_facturas['bandera_porcentaje'] = np.where(
        df_contratos_facturas['porcentaje_raw'].fillna(0) >= 30, # Usar el valor numérico y considerar NaN como 0
        1, # True
        0  # False
    )
    print("Columna 'bandera_porcentaje' calculada (>= 30% de incremento).")
else:
    print("Error: No se pudo calcular 'bandera_porcentaje'. Asegúrate de que 'porcentaje_raw' existe.")
    df_contratos_facturas['bandera_porcentaje'] = 0 # Inicializar con 0 si no se puede calcular


# 5. Mostrar las primeras filas de df_contratos_facturas incluyendo las nuevas columnas.
print("\nDataFrame con columnas 'valor_final', 'porcentaje_raw', 'porcentaje' y 'bandera_porcentaje':")
# Mostrar solo las columnas relevantes para el cálculo y la bandera
cols_to_display = [CAMPO_NUM_CONT_DFRAW, CAMPO_VALOR, 'total_valor_facturas', 'valor_final', 'porcentaje_raw', 'porcentaje', 'bandera_porcentaje']
# Asegurarse de que las columnas existen antes de intentar mostrarlas
cols_to_display_existing = [col for col in cols_to_display if col in df_contratos_facturas.columns]

display(df_contratos_facturas[cols_to_display_existing].head())

# Opcional: mostrar todas las columnas si se prefiere
# display(df_contratos_facturas.head())

Columna 'valor_final' calculada (valor_contrato + total_valor_facturas).
Columna 'porcentaje_raw' calculada como porcentaje de cambio/incremento.
Columna 'porcentaje' formateada como string.
Columna 'bandera_porcentaje' calculada (>= 30% de incremento).

DataFrame con columnas 'valor_final', 'porcentaje_raw', 'porcentaje' y 'bandera_porcentaje':


,numero_del_contrato,valor_contrato,total_valor_facturas,valor_final,porcentaje_raw,porcentaje,bandera_porcentaje
0,CO1.PCCNTR.2995574,84999414,0.0,8.499941e+07,0.0,0.00%,0
1,CO1.PCCNTR.4699854,1188580238,0.0,1.188580e+09,0.0,0.00%,0
2,CO1.PCCNTR.4331688,27413429,27413429.0,5.482686e+07,100.0,100.00%,1
3,CO1.PCCNTR.5120183,18028415,0.0,1.802842e+07,0.0,0.00%,0
4,CO1.PCCNTR.2562840,345000000,0.0,3.450000e+08,0.0,0.00%,0


## Mostrar resultado

### Subtask:
Mostrar las primeras filas del nuevo DataFrame resultante para validación.


**Reasoning**:
Display the first few rows of the dataframe to inspect the newly calculated columns.



In [ ]:
# Display the first 5 rows of the dataframe to inspect the new columns
print("Primeras filas del DataFrame con columnas calculadas:")
display(df_contratos_facturas.head(2000))

Primeras filas del DataFrame con columnas calculadas:


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor,total_valor_facturas,valor_final,porcentaje_raw,porcentaje,bandera_porcentaje
0,Territorial,702435959,SECRETARIA DE EDUCACION DE CALI,890399011,Valle del Cauca,Cali,Modificado,Mínima cuantía,Ejecutar por el sistema de precios unitarios f...,Ejecutar por el sistema de precios unitarios f...,Obra,2021-11-05T00:00:00.000,2021-11-11,2021-12-25,CO1.PCCNTR.2995574,4143.010.26.1.1592-2021,84999414,INGEBLAR SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901087858,0.0,8.499941e+07,0.000000,0.00%,0
1,Territorial,701400236,MUNICIPIO CANDELARIA,891380038,Valle del Cauca,Candelaria,Modificado,Licitación pública Obra Publica,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,Obra,2023-03-21T00:00:00.000,2023-04-20,2024-10-13,CO1.PCCNTR.4699854,203-11-04-001,1188580238,UNION TEMPORAL AMPLIACIONES A&C,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,0,0.0,1.188580e+09,0.000000,0.00%,0
2,Territorial,704247519,ALCALDIA MUNICIPAL DE ANDALUCIA VALLE DEL CAUCA,891900443,Valle del Cauca,Andalucía,En ejecución,Mínima cuantía,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,2022-12-26T00:00:00.000,2022-12-26,2022-12-30,CO1.PCCNTR.4331688,OP-292-2022,27413429,CONSTRUSERVINDUSTRIAL S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900708708,27413429.0,5.482686e+07,100.000000,100.00%,1
3,Nacional,702217274,CORPORACIÓN AUTONOMA REGIONAL DEL VALLE DEL CAUCA,890399002,Valle del Cauca,Cali,terminado,Mínima cuantía,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,Obra,2023-07-05T00:00:00.000,2023-07-12,2023-10-30,CO1.PCCNTR.5120183,CVC 0610 2023,18028415,*TECHNICAL ADJUSTMENTS JASS*,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,14884464,0.0,1.802842e+07,0.000000,0.00%,0
4,Territorial,702892266,CONCEJO DISTRITAL DE SANTIAGO DE CALI,890399011,Valle del Cauca,Cali,Modificado,Selección Abreviada de Menor Cuantía,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,Obra,2021-06-04T00:00:00.000,2021-06-10,2021-12-10,CO1.PCCNTR.2562840,21.1.7.2.9-2021,345000000,CARLOS OLARTE MARTINEZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,18497613,0.0,3.450000e+08,0.000000,0.00%,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Territorial,704246180,HOSPITAL DEPARTAMENTAL PSIQUIATRICO UNIVERSITA...,890304155,Valle del Cauca,Cali,Activo,Contratación régimen especial,OBRAS DE ADECUACIÓN Y MANTENIMIENTO SEGUNDO PI...,OBRAS DE ADECUACIÓN Y MANTENIMIENTO SEGUNDO PI...,Obra,2025-06-19T00:00:00.000,NaT,2025-08-25,CO1.PCCNTR.7992503,GJ-026-2025,574882640,OSCAR GUSTAVO GOMEZ FLOREZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,91259411,0.0,5.748826e+08,0.000000,0.00%,0
1996,Territorial,703475053,ALCALDIA MUNICIPIO DE JAMUNDI,890399046,Valle del Cauca,Jamundí,Modificado,Selección Abreviada de Menor Cuantía,REALIZAR LA OPTIMIZACIÓN Y MEJORAMIENTO DE RED...,REALIZAR LA OPTIMIZACIÓN Y MEJORAMIENTO DE RED...,Obra,2024-12-27T00:00:00.000,2025-02-17,2025-07-16,CO1.PCCNTR.7096130,34-14-03-1164 de 2024,518354100,CONSORCIO ALCANTARILLADO JAMUNDI,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Sin Descripcion,No Definido,0.0,5.183541e+08,0.000000,0.00%,0
1997,Territorial,704063197,SANTIAGO DE CALI DISTRITO ESPECIAL - UNIDA

## Guardar resultado (opcional pero recomendado)

### Subtask:
Guardar el DataFrame `df_contratos_facturas` resultante en un archivo Parquet en Google Drive.


**Reasoning**:
The subtask is to save the resulting DataFrame `df_contratos_facturas` to a Parquet file in Google Drive. This requires defining the output path and using the `to_parquet` method.



In [ ]:
# 1. Define the output path for the Parquet file in Google Drive.
OUTPUT_PATH_PROCESSED = "/content/drive/MyDrive/contratos_facturas_procesado.parquet"

# 2. Use the .to_parquet() method to save the DataFrame.
df_contratos_facturas.to_parquet(OUTPUT_PATH_PROCESSED, index=False)

# 3. Print a confirmation message.
print(f"DataFrame 'df_contratos_facturas' successfully saved to: {OUTPUT_PATH_PROCESSED}")

DataFrame 'df_contratos_facturas' successfully saved to: /content/drive/MyDrive/contratos_facturas_procesado.parquet


## Summary:

### Data Analysis Key Findings

*   The dataset "ibyt-yi2f" (FACTURAS) was successfully identified and configured, with the relevant columns for contract ID and total value found to be "id\_contrato" and "valor\_total", respectively.
*   A total of 2077 unique contract IDs were loaded from the previously saved Parquet file `contratosadicionados.parquet`.
*   Factura data from the API was successfully downloaded for the 2077 contract IDs using batch processing and pagination, resulting in a total of 157,273 raw factura records.
*   The raw factura data was successfully grouped by contract ID, and the "valor\_total" was summed for each contract, resulting in a DataFrame `df_facturas_grouped` with 1,266 rows representing the total value of facturas per contract.
*   The summed factura values were successfully merged with the original contract data, resulting in a DataFrame `df_contratos_facturas` with 2077 rows and 23 columns. Contracts without corresponding facturas had their `total_valor_facturas` set to 0.
*   New columns `valor_final` (sum of `valor_contrato` and `total_valor_facturas`), `porcentaje` (`valor_final` / `valor_contrato`), and `bandera_porcentaje` (1 if `porcentaje` > 1, 0 otherwise) were successfully calculated and added to the DataFrame.
*   The final processed DataFrame `df_contratos_facturas` was successfully saved to a Parquet file at `/content/drive/MyDrive/contratos_facturas_procesado.parquet`.

### Insights or Next Steps

*   Analyze the distribution of the `porcentaje` column to understand the degree to which factura values exceed initial contract values.
*   Investigate contracts where `bandera_porcentaje` is 1 to identify potential patterns or reasons for the total factura value exceeding the initial contract value (e.g., contract modifications, additional services).


In [ ]:
# Contar cuántos registros tienen 'bandera_porcentaje' igual a 1
if 'df_contratos_facturas' in globals() and not df_contratos_facturas.empty and 'bandera_porcentaje' in df_contratos_facturas.columns:
    count_bandera_1 = df_contratos_facturas['bandera_porcentaje'].sum()
    print(f"Número de registros con 'bandera_porcentaje' = 1: {int(count_bandera_1):,}")
elif 'df_contratos_facturas' not in globals():
    print("El DataFrame 'df_contratos_facturas' no existe. Asegúrate de haber ejecutado los bloques anteriores.")
elif df_contratos_facturas.empty:
    print("El DataFrame 'df_contratos_facturas' está vacío.")
else:
    print("La columna 'bandera_porcentaje' no se encuentra en el DataFrame 'df_contratos_facturas'.")

Número de registros con 'bandera_porcentaje' = 1: 400


In [ ]:
import pandas as pd

# Ruta del archivo Parquet procesado
OUTPUT_PATH_PROCESSED = "/content/drive/MyDrive/contratos_facturas_procesado.parquet"

print(f"Cargando DataFrame desde: {OUTPUT_PATH_PROCESSED}")
try:
    df_procesado = pd.read_parquet(OUTPUT_PATH_PROCESSED)
    print("DataFrame cargado exitosamente.")

    # Generar estadísticas descriptivas
    print("\nEstadísticas descriptivas del DataFrame:")
    display(df_procesado.describe())

    # Opcional: Estadísticas descriptivas para columnas específicas si es necesario
    # print("\nEstadísticas descriptivas de columnas clave:")
    # key_cols = [CAMPO_VALOR, 'total_valor_facturas', 'valor_final', 'porcentaje_raw']
    # existing_key_cols = [col for col in key_cols if col in df_procesado.columns]
    # if existing_key_cols:
    #     display(df_procesado[existing_key_cols].describe())


except FileNotFoundError:
    print(f"Error: El archivo {OUTPUT_PATH_PROCESSED} no fue encontrado.")
except Exception as e:
    print(f"Ocurrió un error al leer o procesar {OUTPUT_PATH_PROCESSED}: {e}")

# Liberar memoria del DataFrame cargado si no se necesita inmediatamente
# del df_procesado
# gc.collect()

Cargando DataFrame desde: /content/drive/MyDrive/contratos_facturas_procesado.parquet
DataFrame cargado exitosamente.

Estadísticas descriptivas del DataFrame:


,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,valor_contrato,total_valor_facturas,valor_final,porcentaje_raw,bandera_porcentaje
count,2020,2077,2.077000e+03,2.077000e+03,2.077000e+03,2077.000000,2077.000000
mean,2023-02-28 03:21:01.782178304,2023-10-02 00:37:26.316803328,2.423205e+09,1.284568e+08,2.551662e+09,66.458106,0.192585
min,2017-09-07 00:00:00,2017-12-11 00:00:00,1.398000e+05,0.000000e+00,5.100000e+06,0.000000,0.000000
25%,2022-04-06 06:00:00,2022-12-02 00:00:00,1.577632e+08,0.000000e+00,1.862545e+08,0.000000,0.000000
50%,2023-04-26 12:00:00,2023-12-22 00:00:00,4.974984e+08,0.000000e+00,5.645054e+08,0.000000,0.000000
75%,2024-07-02 00:00:00,2024-12-31 00:00:00,1.358171e+09,0.000000e+00,1.589164e+09,0.000000,0.000000
max,2025-08-01 00:00:00,2026-01-22 00:00:00,1.178148e+11,1.372704e+10,1.178148e+11,98717.879828,1.000000
std,NaN,NaN,7.583089e+09,6.849724e+08,7.643169e+09,2166.032116,0.394425


In [ ]:
import pandas as pd

# Ruta del archivo Parquet procesado
OUTPUT_PATH_PROCESSED = "/content/drive/MyDrive/contratos_facturas_procesado.parquet"

print(f"Cargando DataFrame completo desde: {OUTPUT_PATH_PROCESSED}")
try:
    df_procesado_full = pd.read_parquet(OUTPUT_PATH_PROCESSED)
    print("DataFrame cargado exitosamente.")

    print("\nMostrando el DataFrame completo:")
    # Usar display para una mejor visualización en Colab, que maneja DataFrames grandes
    display(df_procesado_full)

except FileNotFoundError:
    print(f"Error: El archivo {OUTPUT_PATH_PROCESSED} no fue encontrado.")
except Exception as e:
    print(f"Ocurrió un error al leer o procesar {OUTPUT_PATH_PROCESSED}: {e}")

# Liberar memoria del DataFrame cargado si no se necesita inmediatamente
# del df_procesado_full
# gc.collect()

Cargando DataFrame completo desde: /content/drive/MyDrive/contratos_facturas_procesado.parquet
DataFrame cargado exitosamente.

Mostrando el DataFrame completo:


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor,total_valor_facturas,valor_final,porcentaje_raw,porcentaje,bandera_porcentaje
0,Territorial,702435959,SECRETARIA DE EDUCACION DE CALI,890399011,Valle del Cauca,Cali,Modificado,Mínima cuantía,Ejecutar por el sistema de precios unitarios f...,Ejecutar por el sistema de precios unitarios f...,Obra,2021-11-05T00:00:00.000,2021-11-11,2021-12-25,CO1.PCCNTR.2995574,4143.010.26.1.1592-2021,84999414,INGEBLAR SAS,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,901087858,0.00,8.499941e+07,0.000000,0.00%,0
1,Territorial,701400236,MUNICIPIO CANDELARIA,891380038,Valle del Cauca,Candelaria,Modificado,Licitación pública Obra Publica,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,AMPLIACIÓN Y ADECUACIÓN DE LA CASA DELA CULTUR...,Obra,2023-03-21T00:00:00.000,2023-04-20,2024-10-13,CO1.PCCNTR.4699854,203-11-04-001,1188580238,UNION TEMPORAL AMPLIACIONES A&C,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,0,0.00,1.188580e+09,0.000000,0.00%,0
2,Territorial,704247519,ALCALDIA MUNICIPAL DE ANDALUCIA VALLE DEL CAUCA,891900443,Valle del Cauca,Andalucía,En ejecución,Mínima cuantía,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,CONTRATAR POR EL SISTEMA DE PRECIOS UNITARIOS ...,Obra,2022-12-26T00:00:00.000,2022-12-26,2022-12-30,CO1.PCCNTR.4331688,OP-292-2022,27413429,CONSTRUSERVINDUSTRIAL S.A.S.,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,900708708,27413429.00,5.482686e+07,100.000000,100.00%,1
3,Nacional,702217274,CORPORACIÓN AUTONOMA REGIONAL DEL VALLE DEL CAUCA,890399002,Valle del Cauca,Cali,terminado,Mínima cuantía,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,MANTENIMIENTO Y REPARACIÓN DE LOS BIENES MUEBL...,Obra,2023-07-05T00:00:00.000,2023-07-12,2023-10-30,CO1.PCCNTR.5120183,CVC 0610 2023,18028415,*TECHNICAL ADJUSTMENTS JASS*,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,14884464,0.00,1.802842e+07,0.000000,0.00%,0
4,Territorial,702892266,CONCEJO DISTRITAL DE SANTIAGO DE CALI,890399011,Valle del Cauca,Cali,Modificado,Selección Abreviada de Menor Cuantía,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,LLEVAR A CABO POR EL SISTEMA DE PRECIOS UNITAR...,Obra,2021-06-04T00:00:00.000,2021-06-10,2021-12-10,CO1.PCCNTR.2562840,21.1.7.2.9-2021,345000000,CARLOS OLARTE MARTINEZ,https://community.secop.gov.co/Public/Tenderin...,SECOPII,Cédula de Ciudadanía,18497613,0.00,3.450000e+08,0.000000,0.00%,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,Nacional,703940056,SENA REGIONAL VALLE Grupo Administrativo CNPB,899999034,Valle del Cauca,Buenaventura,terminado,Mínima cuantía,Contratar las adecuaciones necesarias para bar...,Contratar las adecuaciones necesarias para bar...,Obra,2024-08-28T00:00:00.000,2024-08-30,2024-11-28,CO1.PCCNTR.6684187,CO1.PCCNTR.6684187,69964516,HECTOR ANGULO SINISTERRA,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,16508178,69964516.54,1.399290e+08,100.000001,100.00%,1
2073,Territorial,712120096,VALLECAUCANA DE AGUAS S.A. E.S.P,900333452,Valle del Cauca,Cali,Modificado,Licitación pública Obra Publica,EJECUTAR EL PLAN DE OBRAS PÚBLICAS BAJO LA MOD...,EJECUTAR EL PLAN DE OBRAS PÚBLICAS BAJO LA MOD...,Obra,2023-12-15T00:00:00.000,2024-02-26,2025-11-04,CO1.PCCNTR.5494497,2000.13.05.003-2023,1498044285,UTR2023,https://community.secop.gov.co/Public/Tenderin...,SECOPII,No Definido,No Definido,0.00,1.498044e+09,0.000000,0.00%,0
2074,Territorial,704246388,ALCALDIA MUNICIPAL DE BUGA,891380033,Valle del Cauca,Guadalajara De 